<a href="https://colab.research.google.com/github/iskra3138/colab_seminar/blob/master/colab%EC%97%90%EC%84%9C_KorQuad_%ED%95%99%EC%8A%B5%ED%95%98%EA%B8%B01_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab에서 GPU로 KoQuAD 학습하기 1
- GPU instance와 Google Drive 활용
- https://github.com/google-research/bert 의 학습 코드 사용
- P100에서 2~3 시간 걸렸던 듯

### 할당받은 GPU 확인

In [0]:
## p100 할당받을 때까지 런타임 초기화 시도 / 안되면 그냥~
!nvidia-smi

### Google Drive Mount

In [0]:
from google.colab import drive
drive.mount('/gdrive') # 'gdrive'라는 이름으로 Google Drive를 Mount하겠다!

- Google Drive의 최상위 Directory는 '위에서 선언한 이름/My Drive'가 됨

### 실험 공간 생성

In [0]:
# 실험을 위한 Directory 생성방법 1

!mkdir '/gdrive/My Drive/KorQuAD'

In [0]:
# 실험을 위한 Directory 생성방법 2
import os
path='/gdrive/My Drive/KorQuAD'
if not os.path.exists(path):
  os.makedirs(path)

In [0]:
# 실험 Directory로 이동 1
cd '/gdrive/My Drive/KorQuAD'

In [0]:
# 실험 Directory로 이동 1
os.chdir(path)

In [0]:
# 현재 directory 확인 1
pwd

In [0]:
# 현재 directory 확인 2
os.getcwd()

### 필요한 파일들 저장하기
- 재 실험 시 실행할 필요 없음

In [0]:
# pre-train된 모델 다운받기
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip

In [0]:
# 압축 풀고 압축파일 삭제
!unzip multi_cased_L-12_H-768_A-12.zip -d pretrained_files
!rm multi_cased_L-12_H-768_A-12.zip

In [0]:
## 압축 해제된 파일 확인
ls pretrained_files/multi_cased_L-12_H-768_A-12/

In [0]:
# KorQuAD Data 다운받기

!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
!wget https://korquad.github.io/dataset/evaluate-v1.0.py

In [0]:
## 파일 확인
ls

In [0]:
# 학습에 필요한 코드 다운받기
!git clone https://github.com/google-research/bert.git ./bert_files

In [0]:
# 코드 확인
ls

In [0]:
# 학습방법 1 : 샘플코드
## 할당된 GPU가 메모리가 작다면(e.g P4) oom에러 발생
!python ./bert_files/run_squad.py \
  --vocab_file=./pretrained_files/multi_cased_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=./pretrained_files/multi_cased_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=./pretrained_files/multi_cased_L-12_H-768_A-12/bert_model.ckpt \
  --do_train=True \
  --train_file=./KorQuAD_v1.0_train.json \
  --do_predict=True \
  --predict_file=./KorQuAD_v1.0_dev.json \
  --train_batch_size=16 \
  --learning_rate=3e-5 \
  --num_train_epochs=2.0 \
  --max_seq_length=256 \
  --doc_stride=128 \
  --output_dir=./temp \
  --do_lower_case=False \

In [0]:
# 학습방법 2 : output이 많아서 크롬 메모리가 걱정되거나, 학습 중에 다른 짓을 하고 싶을 때는 background로 실행
## output이 많고 오래 걸리는 작업이라면, 향후 로그 관리나, 메모리 등등에서 이 방법이 더 나은 듯
!nohup python ./bert_files/run_squad.py \
  --vocab_file=./pretrained_files/multi_cased_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=./pretrained_files/multi_cased_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=./pretrained_files/multi_cased_L-12_H-768_A-12/bert_model.ckpt \
  --do_train=True \
  --train_file=./KorQuAD_v1.0_train.json \
  --do_predict=True \
  --predict_file=./KorQuAD_v1.0_dev.json \
  --train_batch_size=16 \
  --learning_rate=3e-5 \
  --num_train_epochs=2.0 \
  --max_seq_length=256 \
  --doc_stride=128 \
  --output_dir=./temp \
  --do_lower_case=False \
  > ./log_gpu.txt &

In [0]:
## nohub으로 background 학습 중이라면 tail로 모니터링
!tail -f log_gpu.txt

In [0]:
# 학습결과물 확인
ls ./temp

In [0]:
# 성능평가

!python ./evaluate-v1.0.py ./KorQuAD_v1.0_dev.json ./temp/predictions.json

In [0]:
# 구글 드라이브 용량이 부족하면 종료하고 별도로 드라이브 앱에서 휴지통도 비워줄 것!
#rm -r '/gdrive/My Drive/KorQuAD'

### nohup으로 log 파일을 만들었다면 local로 다운로드

In [0]:
from google.colab import files

files.download('./log_gpu.txt')

## Clean Up
Before running the next exercise, run the following cell to terminate the kernel and free memory resources:

In [0]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)